In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import Ipynb_importer
from utils_x import *
from parameters_x import *

all_col = ['unique_symbol', 'date_time', 'close', 'turnover', 'open', 'high', 'low', 'volume',"use_tag", "adjustment", "adjustment_cumsum"]
shift_col = ['close', 'turnover', 'open', 'high', 'low', 'volume', 'wap', "adjustment_cumsum"]

importing Jupyter notebook from utils_x.ipynb
importing Jupyter notebook from parameters_x.ipynb


# clean3 对数据进行 factor处理后 根据clean2标记的tag准备训练数据

In [2]:
print("------------------- load data--------------------")
# data = pd.read_csv('index500_v2.csv', dtype=object)
data = pd.read_csv('index500_v2_adjustment.csv', dtype=object)
data["adjustment"] =  data["adjustment"].fillna(0)


print("------------------- pre data--------------------")
data = data[all_col].copy()
data['turnover'] = data['turnover'].apply(lambda x:np.float(x))
data['volume'] = data['volume'].apply(lambda x:np.float(x))
data['open'] = data['open'].apply(lambda x:np.float(x))
data['close'] = data['close'].apply(lambda x:np.float(x))
data['high'] = data['high'].apply(lambda x:np.float(x))
data['low'] = data['low'].apply(lambda x:np.float(x))
data['adjustment'] = data['adjustment'].apply(lambda x:np.float(x))
data['adjustment_cumsum'] = data['adjustment_cumsum'].apply(lambda x:np.float(x))
print("---------------- pre data step 1-----------------")



------------------- load data--------------------
------------------- pre data--------------------
---------------- pre data step 1-----------------


In [3]:
data["date_time"]  = pd.to_datetime(data["date_time"])

In [4]:
data[(data["unique_symbol"] == "600499")]

,unique_symbol,date_time,close,turnover,open,high,low,volume,use_tag,adjustment,adjustment_cumsum
241,600499,2010-01-29,23.46,109666784.0,23.50,24.10,23.30,4619500.0,1,0.0,0.000000
2249,600499,2010-02-01,24.63,180840480.0,23.31,24.67,23.20,7563900.0,1,0.0,0.000000
3913,600499,2010-02-02,24.14,147885120.0,24.79,25.11,23.69,6094300.0,1,0.0,0.000000
5587,600499,2010-02-03,24.48,154885040.0,24.36,25.90,23.30,6459000.0,1,0.0,0.000000
7260,600499,2010-02-04,23.87,102963520.0,24.47,24.47,23.69,4289600.0,1,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
4552104,600499,2017-12-25,10.91,285138121.0,10.88,11.07,10.68,26194715.0,1,0.0,-15.044615
4555356,600499,2017-12-26,10.93,182643286.0,10.90,10.99,10.75,16819957.0,1,0.0,-15.044615
4558603,600499,2017-12-27,11.15,782294534.0,11.47,11.85,11.12,68338688.0,1,0.0,-15.044615
4561852,600499,2017-12-28,10.87,427856166.0,11.10,11.23,10.77,39072457.0,1,0.0,-15.044615


In [5]:
data[(data["unique_symbol"] == "600499") & (data["date_time"].dt.year == 2010)& (data["date_time"].dt.month == 3)]["adjustment_cumsum"]

27270    0.000000
28946    0.000000
30625    0.000000
32312    0.000000
34004    0.000000
35698    0.000000
37420   -5.714615
39142   -5.714615
40804   -5.714615
42532   -5.714615
44247   -5.714615
45930   -5.714615
47618   -5.714615
49342   -5.714615
51057   -5.714615
52724   -5.714615
Name: adjustment_cumsum, dtype: float64

In [6]:
data['wap'] = data['turnover']/data['volume']
gb = data.groupby(['unique_symbol'])['open']
data['t+1_open'] = gb.transform(lambda x: x.shift(-1))
data['t+2_open'] = gb.transform(lambda x: x.shift(-2))
print("------------- open shift done --------------------")

------------- open shift done --------------------


In [7]:
gb = data.groupby(['unique_symbol'])['adjustment']
data['t+2_adjustment'] = gb.transform(lambda x: x.shift(-2))
data["y_open"] = data["t+2_open"] - data['t+2_adjustment']
data['return'] = data.apply(lambda x: (x['y_open'])/x['t+1_open']-1, axis=1)
print("------------- y data done --------------------")

------------- y data done --------------------


In [8]:
# data[data["unique_symbol"] == "000016"]["open"].mean()

In [9]:
p_col = ['close', 'open', 'high', 'low', 'wap']
v_col = ['turnover', 'volume']

data = data[~(data['return'] > 0.25)].copy()
data = data[~(data['return'] < -0.25)].copy()
data = data[~data['return'].isnull()].copy()


print(len(data))
print("---------------- shift data-------------------")
shift_day = 15
for col in shift_col:
    print(col)
    gb_col = "unique_symbol"
    data = nshift(data, gb_col, col, shift_day)
for col in p_col:
    adjust_col = "adjustment_cumsum"    
    data = norm_pcol(data, col, adjust_col, shift_day)
for col in v_col:
    data = norm_vcol(data, col, shift_day)

4560228
---------------- shift data-------------------
close
turnover
open
high
low
volume
wap
adjustment_cumsum


In [10]:
data.to_csv("index500_v3_adjustment_new.csv")